<p align="center">
  <img src="https://huggingface.co/speakleash/Bielik-7B-Instruct-v0.1/raw/main/speakleash_cyfronet.png">
</p>

W tym notebooku przejdziemy całą ścieżkę - od uruchomienia modelu aż po zwrot danych w formacie JSON. Zaczynajmy! :)

### Przygotowanie środowiska

In [1]:
#%%capture

!pip install accelerate -q
!pip install -i https://pypi.org/simple/ bitsandbytes -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.7 MB/s eta 0:00:00


In [2]:
import gc
import torch

In [3]:
import warnings
warnings.filterwarnings("ignore") # tylko na potrzeby naszego demo ;)

### Inicjalizacja modelu (4 bit)

In [4]:
from transformers import (AutoModelForCausalLM, AutoTokenizer,
                          BitsAndBytesConfig, TextStreamer)

# Wybór urządzenia do uruchomienia modelu
device = "cuda"
model_name = 'speakleash/Bielik-7B-Instruct-v0.1'

# Ustawienie kwantyzacji na 4 bity, aby zmieścić na GPU na Colabie
load_4_bit = True

# Ustawienie tokenizera i kluczowych tokenów
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Obsługa streamingu
streamer = TextStreamer(tokenizer,
                        skip_prompt=True,
                        skip_special_tokens=True)

# Przygotowanie zmiennej pod konfigurację kwantyzacji
quantization_config = None

if load_4_bit:
  quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_compute_dtype=torch.bfloat16)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config)



tokenizer_config.json:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/593 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

### Funkcja pomocnicza do generowania odpowiedzi

In [7]:
# Parametry modelu
temperature = 0.0
max_tokens = 256
top_k = 100
top_p = 1

def generate(prompt: str,
             max_tokens: int = 256,
             temperature: float = 0.0,
             system: str = None,
             return_str: bool = False):

  messages = []

  # Przygotowanie template z promptem
  if system:
    messages.append({"role": "system",
                     "content": system})

  messages.append({"role": "user",
                   "content": prompt})

  input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt")
  model_inputs = input_ids

  # Przerzucenie inputu na GPU, jeśli jest dostępne
  if torch.cuda.is_available():
    model_inputs = input_ids.to(device)

  # Generowanie odpowiedzi przez model
  outputs = model.generate(
      model_inputs,
      streamer = streamer,
      max_new_tokens=max_tokens,
      do_sample=True if temperature else False,
      temperature = temperature,
      top_k = top_k,
      top_p = top_p)

  # Wydrukowanie odpowiedzi modelu bądź zwrotka do zmiennej
  if return_str:
    return tokenizer.batch_decode(outputs, skip_special_tokens=False)

#### Test

In [8]:
# Test generowania tekstu przez model
generate("Hej, jesteś tam?")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Cześć! Tak, jestem tutaj. W czym mogę Ci pomóc?


In [9]:
# Test z promptem systemowym i właściwym
generate(prompt="Hej, czym się zajmujesz?",
         system="Jesteś programistą baz danych, który chce rozwijać swoje kompetencje w obszarze AI")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Cześć! Jako programista baz danych specjalizuję się w projektowaniu, tworzeniu i zarządzaniu systemami, które przechowują i organizują duże zbiory danych. W ostatnim czasie zacząłem interesować się sztuczną inteligencją i uczeniem maszynowym, aby poszerzyć swoje kompetencje i zrozumieć, jak te technologie mogą wpływać na moją dziedzinę. Uczestniczę w kursach i czytam artykuły na temat AI, aby zdobyć wiedzę i umiejętności związane z wykorzystaniem AI w bazach danych. Chciałbym w przyszłości wdrażać rozwiązania AI i machine learning w mojej pracy, aby pomagać organizacjom w lepszym zrozumieniu i analizie ich danych. Czy masz jakieś pytania


# Przykład 1

Załóżmy, że odpowiadasz z klasyfikację tekstów w redakcji i aby mieć kontrolę nad tym procesem chcesz opisać każdy z nich zestawem słów kluczowych bądź zwrotów. Robienie tego w sposób manualny na pewno będzie czasochłonne. Co gdybym Ci powiedział, że można to usprawnić korzystając z naszego Bielika?

Wybrałem pierwszy z brzegu artykuł: https://sport.interia.pl/robert-lewandowski/news-mecz-lewandowskiego-jak-horror-z-happy-endem-fc-barcelona-go,nId,7763629

Najpierw musimy wydobyć tekst z artykułu! Do tego przydadzą się pythonowe biblioteki **requests** oraz **justext**!

In [10]:
# Instalacja niezbędnych bibliotek
!pip install justext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.8/837.8 kB 43.5 MB/s eta 0:00:00


In [13]:
import requests
import justext

url = "https://sport.interia.pl/robert-lewandowski/news-mecz-lewandowskiego-jak-horror-z-happy-endem-fc-barcelona-go,nId,7763629"

def get_article_from_url(url: str):
  # Przygotowanie zmiennej do zbierania fragmentów artykułu ze strony internetowej
  article_parts = []

  # Zebranie odpowiedzi w HTML
  response = requests.get(url)

  # Przetworzenie odpowiedzi przy użyciu biblioteki justext
  paragraphs = justext.justext(response.content, justext.get_stoplist("Polish"))
  for paragraph in paragraphs:
    if not paragraph.is_boilerplate:
      article_parts.append(paragraph.text)

  return " ".join(article_parts)

In [14]:
# Test działania funkcji
get_article_from_url(url)

'Mecz Lewandowskiego jak horror z happy endem, FC Barcelona górą w hicie Gdy dookoła FC Barcelony dzieje się tak wiele w kwestiach organizacyjnych, kibice tej drużyny wolą skupić się na boisku. W sobotę "Blaugrana" podejmowała u siebie Athletic Club, który w minionym sezonie potrafił sprawić jej spore problemy. Mimo tego trener Hansi Flick miał nadzieję na rozpoczęcie sezonu ligowego od dwóch kolejnych zwycięstw. Ta sztuka ostatecznie się udała i jego podopieczni wygrali 2:1, a gola na wagę trzech punktów strzelił nie kto inny, jak Robert Lewandowski. Wcześniej Polak zmarnował kilka dogodnych okazji. FC Barcelona w pierwszej kolejce zdołała uciec obu największym rywalom z Madrytu na dwa punkty. Trener Hansi Flick miał nadzieję na drugie zwycięstwo z rzędu, a rywalem jego nowego zespołu był Athletic Club. Dobra pierwsza połowa "Lewego" - tyle, że bez puenty Sobotnie spotkanie miało wiele podtekstów. Najważniejszy z nich dotyczył osoby Nico Williamsa. Gwiazda Euro 2024 była już przecież 

In [15]:
# Pobranie treści artykułu do zmiennej
article = get_article_from_url(url)

# Przygotowanie promptów
system = 'Jesteś ekspertem od analizy tekstów. Zapoznaj się z treścią podanego artykułu i podaj słowa kluczowe, najlepiej opisujące jego treść.'
prompt = f"Tekst do analizy: {article}"

# Zwiększenie limitu tokenów
max_tokens = 500

# Generowanie odpowiedzi Bielika!
generate(prompt,
         max_tokens=max_tokens)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Analiza:

Mecz piłki nożnej: FC Barcelona vs Athletic Club
Wynik: FC Barcelona 2:1 Athletic Club
Data: 20 sierpnia 2022
Trener FC Barcelona: Hansi Flick
Najlepszy strzelec FC Barcelona w tym sezonie: Robert Lewandowski (1 goal)
Podsumowanie:
Robert Lewandowski strzelił gola w meczu FC Barcelona z Athletic Bilbao, który zakończył się zwycięstwem "Blaugrany" 2:1. Polak otworzył wynik spotkania, ale wcześniej zmarnował kilka dogodnych okazji.
Przebieg meczu:
Pierwsza połowa była wyrównana, z niewielką przewagą FC Barcelona. Gospodarze objęli prowadzenie w 24. minucie po bramce Lamine Yamala, który wykorzystał rzut wolny i błąd obrońcy Athletic Bilbao. Robert Lewandowski miał dwie dobre okazje do zdobycia gola, ale nie zdołał pokonać bramkarza rywali.
W drugiej połowie obraz gry nie uległ zmianie. FC Barcelona dominowała, ale Athletic Bilbao również miało swoje szanse. W 57. minucie Oihan Sancet wykorzystał rzut karny podyktowany za faul na Alexie Berenguera i doprowadził do remisu.
W 70.

### Odpowiedź (na ten moment z Areny)

Na podstawie podanego artykułu, oto słowa kluczowe najlepiej opisujące jego treść:

* FC Barcelona
* Athletic Club
* Robert Lewandowski
* Hansi Flick
* Liga hiszpańska
* Mecz piłkarski
* Gol
* Rzut karny
* Rzut wolny
* Nico Williams
* Lamine Yamal
* Raphinha
* Pedri
* Marc-Andre ter Stegen
* Alex Padilla
* Oihan Sancet
* Transfery piłkarskie
* Kibice
* VAR (Video Assistant Referee)
* Sezon ligowy
* Problemy organizacyjne
* Rywale z Madrytu
* Chronologia
* Kontra
* Fatalna seria
* Obiecujący projekt
* Katalonia
* Pressing
* Okazje bramkowe
* Słupek

Te słowa kluczowe odzwierciedlają główne tematy i wydarzenia opisane w artykule, takie jak mecz piłkarski, udział konkretnych zawodników, aspekty taktyczne, a także kontekst organizacyjny i sportowy FC Barcelony.

# Przykład 2

Wykorzystamy ten sam artykuł, jednak tym razem nieco ograniczymy ramy pracy modelu.

In [17]:
article = get_article_from_url(url)

system = 'Jesteś ekspertem od analizy tekstów. Zapoznaj się z treścią podanego artykułu i podaj słowa kluczowe, najlepiej opisujące jego treść. Słowa kluczowe powinny być hasłami lub nazwami \
  produktów oraz być jak najlepszym odzwierciedleniem potencjalnych reklam, które wyświetlą się użytkownikowi czytającemu artykuł. Nie generuj ich więcej niż 10.'
prompt = f"Tekst do analizy: {article}"
max_tokens = 550

generate(prompt,
         max_tokens=max_tokens)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Analiza:

Mecz piłki nożnej: FC Barcelona vs Athletic Club
Wynik: FC Barcelona 2:1 Athletic Club
Data: 20 sierpnia 2022
Trener FC Barcelona: Hansi Flick
Najlepszy strzelec FC Barcelona w tym sezonie: Robert Lewandowski (1 goal)
Podsumowanie:
Robert Lewandowski strzelił gola w meczu FC Barcelona z Athletic Bilbao, który zakończył się zwycięstwem "Blaugrany" 2:1. Polak otworzył wynik spotkania, ale wcześniej zmarnował kilka dogodnych okazji.
Przebieg meczu:
Pierwsza połowa była wyrównana, z niewielką przewagą FC Barcelona. Gospodarze objęli prowadzenie w 24. minucie po bramce Lamine Yamala, który wykorzystał rzut wolny i błąd obrońcy Athletic Bilbao. Robert Lewandowski miał dwie dobre okazje do zdobycia gola, ale nie zdołał pokonać bramkarza rywali.
W drugiej połowie obraz gry nie uległ zmianie. FC Barcelona dominowała, ale Athletic Bilbao również miało swoje szanse. W 57. minucie Oihan Sancet wykorzystał rzut karny podyktowany za faul na Alexie Berenguera i doprowadził do remisu.
W 70.

### Odpowiedź Bielika (z Areny)

Słowa kluczowe oparte na podanym artykule:

1. FC Barcelona
2. Robert Lewandowski
3. Hansi Flick
4. Athletic Club
5. Nico Williams
6. La Liga
7. Mecz
8. Gol
9. Rzut wolny
10. Reklama

Te słowa kluczowe odzwierciedlają główne tematy artykułu, takie jak drużyny, kluczowi zawodnicy, wydarzenia sportowe oraz elementy, które mogłyby być wykorzystane w reklamach związanych z tym artykułem.

# Przykład 3

Teraz Bielik zostanie poproszony o nieco bardziej ustrukturyzowaną odpowiedź. Zobaczcie sami...

In [18]:
article = get_article_from_url(url)

system = 'Jesteś ekspertem od analizy tekstów. Zapoznaj się z treścią podanego artykułu i podaj słowa kluczowe, najlepiej opisujące jego treść. Słowa kluczowe powinny być hasłami lub nazwami \
  produktów oraz być jak najlepszym odzwierciedleniem potencjalnych reklam, które wyświetlą się użytkownikowi czytającemu artykuł. Nie generuj ich więcej niż 10. Wyniki zwróć w formacie obiektu JSON z \
  kluczami: keywords, summary oraz famous_players.'
prompt = f"Tekst do analizy: {article}"
max_tokens = 550

generate(prompt,
         max_tokens=max_tokens)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Analiza:

Mecz piłki nożnej: FC Barcelona vs Athletic Club
Wynik: FC Barcelona 2:1 Athletic Club
Data: 20 sierpnia 2022
Trener FC Barcelona: Hansi Flick
Najlepszy strzelec FC Barcelona w tym sezonie: Robert Lewandowski (1 goal)
Podsumowanie:
Robert Lewandowski strzelił gola w meczu FC Barcelona z Athletic Bilbao, który zakończył się zwycięstwem "Blaugrany" 2:1. Polak otworzył wynik spotkania, ale wcześniej zmarnował kilka dogodnych okazji.
Przebieg meczu:
Pierwsza połowa była wyrównana, z niewielką przewagą FC Barcelona. Gospodarze objęli prowadzenie w 24. minucie po bramce Lamine Yamala, który wykorzystał rzut wolny i błąd obrońcy Athletic Bilbao. Robert Lewandowski miał dwie dobre okazje do zdobycia gola, ale nie zdołał pokonać bramkarza rywali.
W drugiej połowie obraz gry nie uległ zmianie. FC Barcelona dominowała, ale Athletic Bilbao również miało swoje szanse. W 57. minucie Oihan Sancet wykorzystał rzut karny podyktowany za faul na Alexie Berenguera i doprowadził do remisu.
W 70.

Na podstawie podanego artykułu, oto słowa kluczowe w formacie JSON:

```json
{
  "keywords": [
    "FC Barcelona",
    "Athletic Club",
    "Robert Lewandowski",
    "Hansi Flick",
    "Nico Williams",
    "Lamine Yamal",
    "Raphinha",
    "Pedri",
    "La Liga",
    "Euro 2024"
  ],
  "summary": "FC Barcelona pokonuje Athletic Club 2:1 w meczu La Liga, dzięki zwycięskiej bramce Roberta Lewandowskiego. Mecz był pełen emocji i okazji bramkowych, a Barcelona imponowała kreowaniem szans i pracą w pressingu.",
  "famous_players": [
    "Robert Lewandowski",
    "Nico Williams",
    "Lamine Yamal",
    "Raphinha",
    "Pedri"
  ]
}
```

Słowa kluczowe zostały wybrane na podstawie najważniejszych tematów i osób wymienionych w artykule, które mogłyby być użyte w reklamach lub nagłówkach związanych z tym wydarzeniem sportowym.

Sami oceńcie, jak Bielik poradził sobie z postawionymi przed nim zadaniami! :)